 # EDA using Plotly 
1. Countrywise
    1. Average Rating 
    2.  Number of Resturants 
2. By Cuisine  
3. Is there any Pattern in Names 
4. Indian Perspective 
   1. City wise Aggregate Rating 
   2. City wise Number of Resturants 
   3. What's Popular in 
        1.  New Delhi
        2. Noida
        3. Gurgaon 
5. Where are these resturants 


**Importing all Library **

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib as mpl
import pylab
import seaborn as sns
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**Reading Data**

In [8]:
df = pd.read_csv('../input/zomato.csv',encoding = "ISO-8859-1")
Country = pd.read_excel('../input/Country-Code.xlsx')



# Getting Country Names in our Data 

In [9]:
data = pd.merge(df, Country,on='Country Code')
data.head()

**Initializing Plotly **

In [10]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()
import plotly.graph_objs as go


# Where are the resturants registered on Zomato  ?

In [11]:
Temp = data['Country'].value_counts()
data1 = [go.Choropleth(
    locationmode='country names',
    locations=Temp.index.values,
    text=Temp.index,
    z=Temp.values
)]
layout = go.Layout(
    title='Number of Resturants Registered on Zomato by Country',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)


In [12]:

data1 = [go.Bar(x=Temp.index,
            y=Temp.values)]
layout = go.Layout(
    title='Number of Resturants Registered on Zomato by Country',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

**As we can see most of them are in India with over 8000 of them out of total 9551**



# How do different countries rate on Zomato 

In [13]:
Temp=(data.groupby(['Country'], as_index=False)['Aggregate rating'].mean())

data1 = ([go.Choropleth(
    locationmode='country names',
    locations=Temp['Country'],
    text=Temp['Country'],
    z=Temp['Aggregate rating']
)])
layout = go.Layout(
    title='Average Rating on Zomato by Country',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)


In [14]:

data1 = ([go.Bar(x=Temp['Country'],
    y=Temp['Aggregate rating'])])
layout = go.Layout(
    title='Average on Zomato by Country',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

# What Are The Popular Cusines on Zomato 
Lets just take 20 most Common  cuisines in the dataset for now 

In [15]:
Cusine_data=(data.groupby(['Cuisines'], as_index=False)['Restaurant ID'].count())
Cusine_data.columns = ['Cuisines', 'Number of Resturants']
Cusine_data['Mean Rating']=(data.groupby(['Cuisines'], as_index=False)['Aggregate rating'].mean())['Aggregate rating']
#Cusine_data.sort_values(['Number of Resturants'],ascending=False).head(20)
TwentyMostPopularCusines = (Cusine_data.sort_values(['Number of Resturants'],ascending=False).head(20))['Cuisines']
Top20 = Cusine_data.sort_values(['Number of Resturants'],ascending=False).head(20)
Cusine_data.sort_values(['Number of Resturants'],ascending=False).head(20)

In [16]:
data1 = ([go.Bar(x=Top20['Cuisines'],
    y=Top20['Number of Resturants'])])
layout = go.Layout(
    title='Number of Resturants on Zomato by Cuisines',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

In [17]:
DataForTwentyMostPopularCusines = (data[data['Cuisines'].isin(TwentyMostPopularCusines)])
TwentyMostPopularCusines= TwentyMostPopularCusines.reset_index()['Cuisines']

In [18]:
a=[None] * 20
b= [None] * 20
trace = [None] * 20
for i in range (0,20):
    TwentyMostPopularCusines[i]
    a[i]= DataForTwentyMostPopularCusines[DataForTwentyMostPopularCusines['Cuisines']==TwentyMostPopularCusines[i]]['Aggregate rating']
    b[i] =a[i].reset_index()['Aggregate rating']
    trace[i] = go.Box(
    y=b[i],
    name = TwentyMostPopularCusines[i],
        boxmean=True
    )

In [19]:

data1 = trace
layout = go.Layout(
    title='How are these 20 Cuisines rated ',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

# Is there a pattern in Names 

In [20]:
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS

#mpl.rcParams['figure.figsize']=(8.0,6.0)    #(6.0,4.0)
             #72 


stopwords = set(STOPWORDS)
wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_font_size=200, 
                         ).generate((data['Restaurant Name']).to_string())

print(wordcloud)
fig = plt.figure(1,figsize=(15, 15))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

**I'll need to do Further study into that **

# ****Diving into India , as India has maximum data in Zomato Dataset ****

In [21]:
Data_India = data[data['Country']=='India']

# How many Resturants are registered on Zomato per city 

In [22]:
Resturants_India = (Data_India.groupby(['City'], as_index=False)['Aggregate rating'].count()).sort_values(by='Aggregate rating',ascending=False)
Resturants_India.columns = ['City', 'Number of Resturants']

trace0 = go.Bar(
    x=Resturants_India['City'],
    y=Resturants_India['Number of Resturants'],
    )

data1 = [trace0]
layout = go.Layout(
    title='Number of Resturants on Zomato by City',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

# How different cities rate on average

In [23]:
Resturants_India=(Data_India.groupby(['City'], as_index=False)['Aggregate rating'].mean()).sort_values(by='Aggregate rating',ascending=False)


trace0 = go.Bar(
    x=Resturants_India['City'],
    y=Resturants_India['Aggregate rating'],
    )

data1 = [trace0]
layout = go.Layout(
    title='Average Rating on Zomato by City',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

# What's the popular Cuisine in the 3 Cities with greater than 500 Resturants 

In [24]:
Delhi_Data = data[data['City']=='New Delhi']
Ggn_Data = data[data['City']=='Gurgaon']
Noida_Data =data[data['City']=='Noida']

In [25]:
Delhi_Data=(Delhi_Data.groupby(['Cuisines'])['Aggregate rating'].agg(['mean', 'count'])).reset_index()
Delhi_Data = Delhi_Data[Delhi_Data['count']>10]
Delhi_Data= Delhi_Data.sort_values(by='mean',ascending=False)
trace0 = go.Bar(
    x=Delhi_Data['Cuisines'],
        y=Delhi_Data['mean'],
    )

data1 = [trace0]
layout = go.Layout(
    title='Popular Cuisines in Delhi',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)



In [26]:
Ggn_Data=(Ggn_Data.groupby(['Cuisines'])['Aggregate rating'].agg(['mean', 'count'])).reset_index()
Ggn_Data = Ggn_Data[Ggn_Data['count']>10]
Ggn_Data= Ggn_Data.sort_values(by='mean',ascending=False)
trace0 = go.Bar(
    x=Ggn_Data['Cuisines'],
        y=Ggn_Data['mean'],
    )

data1 = [trace0]
layout = go.Layout(
    title='Popular Cuisines in Gurgaon',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

In [27]:
Noida_Data=(Noida_Data.groupby(['Cuisines'])['Aggregate rating'].agg(['mean', 'count'])).reset_index()
Noida_Data = Noida_Data[Noida_Data['count']>10]
Noida_Data= Noida_Data.sort_values(by='mean',ascending=False)
trace0 = go.Bar(
    x=Noida_Data['Cuisines'],
        y=Noida_Data['mean'],
    )

data1 = [trace0]
layout = go.Layout(
    title='Popular Cuisines in Noida',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

In [28]:
data.head()

# Where are all these resturants : Just having fun with Plotly and Resturant Coordinates :D 

In [40]:
data1 = [dict(
    type='scattergeo',
    lon = data['Longitude'],
    lat = data['Latitude'],
    text = data['Restaurant Name'],
    mode = 'markers',
    marker = dict(
    cmin = 0,
    color = data['Price range'],
    cmax = data['Price range'].max(),
    colorbar=dict(
                title="Price Range"
            )
    )
    
)]
layout = dict(
    title = 'Where are all these Resturants',
    hovermode='closest',
    geo = dict(showframe=False, countrywidth=1, showcountries=True,
               showcoastlines=True, projection=dict(type='Mercator'))
)
fig = go.Figure(data=data1, layout=layout)
iplot(fig)